In [1]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
# load the data
df = pd.read_csv('cc_approval.csv', header=None);
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [3]:
df.columns=['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16']
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [4]:
cols_with_unknowns = ['A1','A2','A4','A5','A6','A7','A14'] #given in the attributes details
cols_with_unknowns_continuous = ['A2', 'A14']
cols_with_unknowns_categorical = ['A1','A4','A5','A6','A7']
for col in cols_with_unknowns_continuous:
    df[col]=df[col].replace('?',np.NaN)
    #Filling missing values with medians of the columns
    df[col] = df[col].fillna(df[col].median())
    
for col in cols_with_unknowns_categorical:
    df[col]=df[col].replace('?',np.NaN)
    #Max fill function for categorical columns
    df[col].fillna(df[col].value_counts().idxmax(), inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      690 non-null    object 
 1   A2      690 non-null    object 
 2   A3      690 non-null    float64
 3   A4      690 non-null    object 
 4   A5      690 non-null    object 
 5   A6      690 non-null    object 
 6   A7      690 non-null    object 
 7   A8      690 non-null    float64
 8   A9      690 non-null    object 
 9   A10     690 non-null    object 
 10  A11     690 non-null    int64  
 11  A12     690 non-null    object 
 12  A13     690 non-null    object 
 13  A14     690 non-null    object 
 14  A15     690 non-null    int64  
 15  A16     690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


In [5]:
df.A2 = df.A2.astype(float)
df.A14 = df.A14.astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   A1      690 non-null    object 
 1   A2      690 non-null    float64
 2   A3      690 non-null    float64
 3   A4      690 non-null    object 
 4   A5      690 non-null    object 
 5   A6      690 non-null    object 
 6   A7      690 non-null    object 
 7   A8      690 non-null    float64
 8   A9      690 non-null    object 
 9   A10     690 non-null    object 
 10  A11     690 non-null    int64  
 11  A12     690 non-null    object 
 12  A13     690 non-null    object 
 13  A14     690 non-null    float64
 14  A15     690 non-null    int64  
 15  A16     690 non-null    object 
dtypes: float64(4), int64(2), object(10)
memory usage: 86.4+ KB


In [6]:
cols_with_continuous = ['A2','A3','A8','A11','A14','A15'] #given in the attributes details

#Capping the outlier rows with Percentiles
for col in cols_with_continuous:
    upper_lim = df[col].quantile(.95)
    lower_lim = df[col].quantile(.05)
    df.loc[(df[col] > upper_lim),col] = upper_lim
    df.loc[(df[col] < lower_lim),col] = lower_lim
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.830,0.165,u,g,w,v,1.25,t,t,1.0,f,g,202.0,0.0,+
1,a,56.231,4.460,u,g,q,h,3.04,t,t,6.0,f,g,43.0,560.0,+
2,a,24.500,0.500,u,g,q,h,1.50,t,f,0.0,f,g,280.0,824.0,+
3,b,27.830,1.540,u,g,w,v,3.75,t,t,5.0,t,g,100.0,3.0,+
4,b,20.170,5.625,u,g,w,v,1.71,t,f,0.0,f,s,120.0,0.0,+


In [7]:
cols_with_cateorical = ['A1','A4','A5','A6','A7','A9','A10','A12','A13'] #given in the attributes details
#One Hot Encoding
for col in cols_with_cateorical:
    encoded_columns = pd.get_dummies(df[col], prefix=col+'_')
    df = df.join(encoded_columns).drop(col, axis=1)

A16 = {'+':1,'-':0}
df.A16 = [A16[binary_val] for binary_val in df.A16]

In [8]:
df.head()

,A2,A3,A8,A11,A14,A15,A16,A1__a,A1__b,A4__l,...,A7__z,A9__f,A9__t,A10__f,A10__t,A12__f,A12__t,A13__g,A13__p,A13__s
0,30.830,0.165,1.25,1.0,202.0,0.0,1,0,1,0,...,0,0,1,0,1,1,0,1,0,0
1,56.231,4.460,3.04,6.0,43.0,560.0,1,1,0,0,...,0,0,1,0,1,1,0,1,0,0
2,24.500,0.500,1.50,0.0,280.0,824.0,1,1,0,0,...,0,0,1,1,0,1,0,1,0,0
3,27.830,1.540,3.75,5.0,100.0,3.0,1,0,1,0,...,0,0,1,0,1,0,1,1,0,0
4,20.170,5.625,1.71,0.0,120.0,0.0,1,0,1,0,...,0,0,1,1,0,1,0,0,0,1


In [9]:
df.shape

(690, 47)

In [10]:
target = df.filter(['A16'], axis=1)
x_inputs = df.drop(['A16'], axis = 1) 

In [11]:
x_inputs.head()

,A2,A3,A8,A11,A14,A15,A1__a,A1__b,A4__l,A4__u,...,A7__z,A9__f,A9__t,A10__f,A10__t,A12__f,A12__t,A13__g,A13__p,A13__s
0,30.830,0.165,1.25,1.0,202.0,0.0,0,1,0,1,...,0,0,1,0,1,1,0,1,0,0
1,56.231,4.460,3.04,6.0,43.0,560.0,1,0,0,1,...,0,0,1,0,1,1,0,1,0,0
2,24.500,0.500,1.50,0.0,280.0,824.0,1,0,0,1,...,0,0,1,1,0,1,0,1,0,0
3,27.830,1.540,3.75,5.0,100.0,3.0,0,1,0,1,...,0,0,1,0,1,0,1,1,0,0
4,20.170,5.625,1.71,0.0,120.0,0.0,0,1,0,1,...,0,0,1,1,0,1,0,0,0,1


In [12]:
target.head()

,A16
0,1
1,1
2,1
3,1
4,1


In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.layers import Dense
from keras.regularizers import l2
from keras.regularizers import l1

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state= 5)

In [14]:
def best_model_L1_lambda2_reg(activation_function = 'sigmoid', no_features = 46, no_neurons_hidden_1 = 30, no_neurons_hidden_2 = 15):
    # create model
    model = Sequential()
    model.add(Dense(no_neurons_hidden_1, input_dim=no_features, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(no_neurons_hidden_2, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=l1(0.01), activation=activation_function))
    # Compile model. We use the the mean square error loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [15]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=best_model_L1_lambda2_reg, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, x_inputs, target, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.49% (2.88%)


In [16]:
def best_model_L1_lambda3_reg(activation_function = 'sigmoid', no_features = 46, no_neurons_hidden_1 = 30, no_neurons_hidden_2 = 15):
    # create model
    model = Sequential()
    model.add(Dense(no_neurons_hidden_1, input_dim=no_features, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(no_neurons_hidden_2, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=l1(0.001), activation=activation_function))
    # Compile model. We use the the mean square error loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [17]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=best_model_L1_lambda3_reg, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, x_inputs, target, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 85.36% (2.65%)


In [18]:
def best_model_L1_lambda4_reg(activation_function = 'sigmoid', no_features = 46, no_neurons_hidden_1 = 30, no_neurons_hidden_2 = 15):
    # create model
    model = Sequential()
    model.add(Dense(no_neurons_hidden_1, input_dim=no_features, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(no_neurons_hidden_2, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=l1(0.0001), activation=activation_function))
    # Compile model. We use the the mean square error loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [19]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=best_model_L1_lambda4_reg, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, x_inputs, target, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.93% (2.57%)


In [20]:
def best_model_L1_lambda1_reg(activation_function = 'sigmoid', no_features = 46, no_neurons_hidden_1 = 30, no_neurons_hidden_2 = 15):
    # create model
    model = Sequential()
    model.add(Dense(no_neurons_hidden_1, input_dim=no_features, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(no_neurons_hidden_2, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=l1(0.1), activation=activation_function))
    # Compile model. We use the the mean square error loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [21]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=best_model_L1_lambda1_reg, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, x_inputs, target, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.20% (4.16%)


In [22]:
def best_model_L2_lambda1_reg(activation_function = 'sigmoid', no_features = 46, no_neurons_hidden_1 = 30, no_neurons_hidden_2 = 15):
    # create model
    model = Sequential()
    model.add(Dense(no_neurons_hidden_1, input_dim=no_features, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(no_neurons_hidden_2, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=l2(0.1), activation=activation_function))
    # Compile model. We use the the mean square error loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [23]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=best_model_L2_lambda1_reg, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, x_inputs, target, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.49% (4.09%)


In [24]:
def best_model_L2_lambda2_reg(activation_function = 'sigmoid', no_features = 46, no_neurons_hidden_1 = 30, no_neurons_hidden_2 = 15):
    # create model
    model = Sequential()
    model.add(Dense(no_neurons_hidden_1, input_dim=no_features, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(no_neurons_hidden_2, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=l2(0.01), activation=activation_function))
    # Compile model. We use the the mean square error loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [25]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=best_model_L2_lambda2_reg, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, x_inputs, target, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.93% (2.40%)


In [26]:
def best_model_L2_lambda3_reg(activation_function = 'sigmoid', no_features = 46, no_neurons_hidden_1 = 30, no_neurons_hidden_2 = 15):
    # create model
    model = Sequential()
    model.add(Dense(no_neurons_hidden_1, input_dim=no_features, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(no_neurons_hidden_2, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=l2(0.001), activation=activation_function))
    # Compile model. We use the the mean square error loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [27]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=best_model_L2_lambda3_reg, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, x_inputs, target, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 85.65% (2.84%)


In [28]:
def best_model_L2_lambda4_reg(activation_function = 'sigmoid', no_features = 46, no_neurons_hidden_1 = 30, no_neurons_hidden_2 = 15):
    # create model
    model = Sequential()
    model.add(Dense(no_neurons_hidden_1, input_dim=no_features, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(no_neurons_hidden_2, kernel_initializer='normal', activation=activation_function))
    model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=l2(0.0001), activation=activation_function))
    # Compile model. We use the the mean square error loss function, and the Adam gradient optimizer.
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [29]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=best_model_L2_lambda4_reg, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
results = cross_val_score(pipeline, x_inputs, target, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 84.93% (2.77%)
